In [1]:
import os
import sys
import subprocess
import numpy as np
import itk
# Add the 'src' folder to the Python path
sys.path.append(os.path.abspath('../'))
from src.DELAT_utils import match_h5_files_by_channels, setup_logging, collect_region_stats_paths
from src.registration import register_and_transform
from src.stats import compute_region_stats
import logging

In [3]:
animal = 'ANM550749_left_JF552'
base_dir = r'X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains'

# Set the parameters manually here

fx_path =  r'X:\Boaz\I2\atlas10_hemi.tif'
param_files_dir = r'X:\Boaz\I2\itk'
annotation_path =  r'X:\Boaz\I2\annotation_10_hemi.nii'

In [4]:
logger = setup_logging(base_dir, animal)
# Load the fixed image (fx)
logger.info(f"Loading fixed image (fx) from {fx_path}.")
fx = itk.imread(fx_path, pixel_type=itk.US)

# Load the parameter files
param_files = [
    os.path.join(param_files_dir, 'Order1_Par0000affine.txt'),
    os.path.join(param_files_dir, 'Order3_Par0000bspline.txt'),
    os.path.join(param_files_dir, 'Order4_Par0000bspline.txt'),
    os.path.join(param_files_dir, 'Order5_Par0000bspline.txt')
]
logger.info(f"Loaded parameter files from {param_files_dir}.")



2024-11-13 13:24:51 - INFO - Logging initialized for animal processing.
2024-11-13 13:24:51 - INFO - Loading fixed image (fx) from X:\Boaz\I2\atlas10_hemi.tif.
2024-11-13 13:25:10 - INFO - Loaded parameter files from X:\Boaz\I2\itk.


In [5]:
# Load the annotation volume
logger.info(f"Loading annotation volume from {annotation_path}.")
itk_annotation = itk.imread(annotation_path, itk.ULL)
annotation_np = itk.array_view_from_image(itk_annotation)
annotation_np.dtype, annotation_np.shape


2024-11-13 13:25:10 - INFO - Loading annotation volume from X:\Boaz\I2\annotation_10_hemi.nii.


(dtype('uint64'), (800, 1320, 658))

In [6]:
# Match H5 files by channels for all animals
logger.info(f"Matching H5 files in {base_dir}.")
animals_files = match_h5_files_by_channels(base_dir)
animals_files

2024-11-13 13:26:04 - INFO - Matching H5 files in X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains.


{'ANM549057_left_JF552': {'ch0': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM549057_left_JF552\\uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM549057_left_JF552\\uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch2': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM549057_left_JF552\\uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'},
 'ANM550749_left_JF552': {'ch0': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch2': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'},
 'ANM550751_left_JF673': {'ch0': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550751_lef

In [7]:
files = animals_files.get(animal)
files

{'ch0': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
 'ch1': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
 'ch2': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'}

In [10]:
output_dir = os.path.join(base_dir, animal, 'itk2')  # Ensure logs and outputs go to /itk
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
logger.info(f"Using output dir: {output_dir}")

2024-11-13 13:37:34 - INFO - Using output dir: X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains\ANM550749_left_JF552\itk2


In [11]:
# Step 2: Apply the generated transforms to all channels (ch0, ch1, ch2)
param_files_generated = [f'TransformParameters.{i}.txt' for i in range(4)]

# Create a new ParameterObject for transformix
parameter_object_transformix = itk.ParameterObject.New()
for p in param_files_generated:
    parameter_path = os.path.join(output_dir, p)
    if os.path.exists(parameter_path):
        parameter_object_transformix.AddParameterFile(parameter_path)
    else:
        logger.warning(f"Warning: Parameter file {parameter_path} not found.")

2024-11-13 13:37:37 - WARNING - Warning: Parameter file X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains\ANM550749_left_JF552\itk2\TransformParameters.0.txt not found.
2024-11-13 13:37:37 - WARNING - Warning: Parameter file X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains\ANM550749_left_JF552\itk2\TransformParameters.1.txt not found.
2024-11-13 13:37:37 - WARNING - Warning: Parameter file X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains\ANM550749_left_JF552\itk2\TransformParameters.2.txt not found.
2024-11-13 13:37:37 - WARNING - Warning: Parameter file X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains\ANM550749_left_JF552\itk2\TransformParameters.3.txt not found.


In [12]:
num_cores = os.cpu_count()  
itk.MultiThreaderBase.SetGlobalDefaultNumberOfThreads(num_cores)
logger.info(f"Register_and_transform started, set ITK to use {num_cores} threads.")

2024-11-13 13:37:49 - INFO - Register_and_transform started, set ITK to use 96 threads.


In [14]:

from DELAT_utils import read_h5_image

In [15]:
mv = read_h5_image(files['ch0'], 'Data')

In [16]:
parameter_object = itk.ParameterObject.New()
for p in param_files:
    parameter_object.AddParameterFile(p)

logger.info("param files read")
# Create output directory if it doesn't exist
if not os.path.isdir(output_dir):
    os.makedirs(output_dir, exist_ok=True)
    logger.info(f"Created output directory {output_dir}")

# Perform registration (this will create TransformParameters.0.txt, etc.)
res, params = itk.elastix_registration_method(fx, mv, parameter_object, log_to_file=True, output_directory=output_dir)

2024-11-13 13:38:25 - INFO - param files read


In [29]:
reference_image =  itk.image_from_array(mv)
reference_image

<itk.itkImagePython.itkImageUS3; proxy of <Swig Object of type 'itkImageUS3 *' at 0x00000290C7F7EE80> >

In [ ]:
reference_image.SetSpacing([10.0, 10.0, 10.0])  # Replace with actual spacing if known
reference_image.SetOrigin([0.0, 0.0, 0.0])
reference_image.SetDirection(itk.Matrix[itk.D, 3, 3]())  # Identity matrix for direction

In [ ]:
itk

In [48]:
transformix_filter = itk.TransformixFilter[itk.Image[itk.F, 3]].New()
transformix_filter.SetTransformParameterObject(params)
transformix_filter.SetMovingImage(reference_image)
transformix_filter.SetOutputDirectory(output_dir)
transformix_filter.SetLogToConsole(True)
transformix_filter.UpdateLargestPossibleRegion()
transformix_filter.SetOutputDirectory(output_dir)
transformix_filter.Update()

RuntimeError: D:\a\im\build\cp311-abi3-win_amd64\_deps\elx-src\Core\Main\itkTransformixFilter.hxx:379:
ITK ERROR: No entry Spacing found in transformParameterMap

In [41]:
itk.TransformixFilter.GetTypes()

<itkTemplate itk::TransformixFilter>
Options:
  [<class 'itk.itkImagePython.itkImageD2'>,]
  [<class 'itk.itkImagePython.itkImageD3'>,]
  [<class 'itk.itkImagePython.itkImageD4'>,]
  [<class 'itk.itkImagePython.itkImageF2'>,]
  [<class 'itk.itkImagePython.itkImageF3'>,]
  [<class 'itk.itkImagePython.itkImageF4'>,]
  [<class 'itk.itkImagePython.itkImageSS2'>,]
  [<class 'itk.itkImagePython.itkImageSS3'>,]
  [<class 'itk.itkImagePython.itkImageSS4'>,]
  [<class 'itk.itkImagePython.itkImageUC2'>,]
  [<class 'itk.itkImagePython.itkImageUC3'>,]
  [<class 'itk.itkImagePython.itkImageUC4'>,]
  [<class 'itk.itkImagePython.itkImageUS2'>,]
  [<class 'itk.itkImagePython.itkImageUS3'>,]
  [<class 'itk.itkImagePython.itkImageUS4'>,]


In [42]:
transformix_filter

<itk.itkTransformixFilterPython.itkTransformixFilterISS2; proxy of <Swig Object of type 'itkTransformixFilterISS2 *' at 0x00000290C86B2430> >